# CNN / RNN / CNN-RNN Hybrid Implementations for human activity recognition

This noteboook was used to compare the effectiveness of using a CNN, an RNNs and a CNN-RNN hybrid aproach for human activity recognition. This implementation uses tensorflow and tensorflow lite. An experimental build of tensorflow(tf-nightly) was used for this research to support LTSM cells on smartphones.

#### Imports

In [ ]:
from __future__ import print_function

# This is important!
import os
os.environ["TF_ENABLE_CONTROL_FLOW_V2"] = "1"

import numpy as np
import pandas as pd

import tensorflow as tf
from tensorflow.keras.layers import Dense, Dropout, Flatten, Reshape, GlobalAveragePooling1D
from tensorflow.keras.layers import Conv1D, MaxPooling1D, LSTM, TimeDistributed, Flatten, Lambda
from tensorflow.keras.layers import Input, Lambda
from tensorflow.keras import utils
from tensorflow.keras.callbacks import Callback
from tensorflow.keras.optimizers import Adam

## Data Set
The researchers opted use and already existing data set from [[1]](#References)


In [ ]:
def read_experiments(labels_path):
    
    exp_path = 'HAPT Data Set/RawData/'
    
    exp_df = read_labels(labels_path)
    acc_data_list_x = []
    acc_data_list_y = []
    acc_data_list_z = []

    gyro_data_list_x = []
    gyro_data_list_y = []
    gyro_data_list_z = []

    
    for index, row in exp_df.iterrows():
        exp_id = format(row['experiment ID'],'02d')
        user_id = format(row['user ID'],'02d')
        start = row['start']
        end = row['end']
        
        acc_exp_path = exp_path + 'acc_exp' + exp_id + '_user' + user_id + '.txt'
        gyro_exp_path = exp_path + 'gyro_exp' + exp_id + '_user' + user_id + '.txt'
        
        acc_data = pd.read_csv(acc_exp_path, delimiter = ' ', header = None, names = ['x','y','z'])
        gyro_data = pd.read_csv(gyro_exp_path, delimiter = ' ', header = None, names = ['x','y','z'])
        
        acc_data_list_x.append(acc_data['x'][start:end].values)
        acc_data_list_y.append(acc_data['y'][start:end].values)
        acc_data_list_z.append(acc_data['z'][start:end].values)
        
        gyro_data_list_x.append(gyro_data['x'][start:end].values)
        gyro_data_list_y.append(gyro_data['y'][start:end].values)
        gyro_data_list_z.append(gyro_data['z'][start:end].values)
        
    exp_df['acc x'] = acc_data_list_x
    exp_df['acc y'] = acc_data_list_y
    exp_df['acc z'] = acc_data_list_z
    exp_df['gyro x'] = gyro_data_list_x
    exp_df['gyro y'] = gyro_data_list_y
    exp_df['gyro z'] = gyro_data_list_z
    
    return exp_df

def read_labels(file_path):

    column_names = [
        'experiment ID',
        'user ID',
        'activity ID',
        'start',
        'end'

    ]

    labels_df = pd.read_csv(file_path, delimiter=' ', header=None, names=column_names);

    return labels_df


## Segmentation
The time series data for each activity was segmented into 'windows' where each window was used as a sample together with its label.

In [ ]:
def segment_experiments(exp_df,segment_length,step): 
    #for x,y,z (acc) and x,y,z gyro
    NUM_FEATURES = 3
    features = ['acc x','acc y', 'acc z', 'gyro x', 'gyro y', 'gyro z']
    
    segments = []
    labels = []
    
    for index, row in exp_df.iterrows():
        for i in range(0 , row['end'] - row['start'], step):
            xa = row['acc x'][i : i + segment_length]
            ya = row['acc y'][i : i + segment_length]
            za = row['acc z'][i : i + segment_length]
#             xg = row['gyro x'][i : i + segment_length]
#             yg = row['gyro y'][i : i + segment_length]
#             zg = row['gyro z'][i : i + segment_length]
            
            if len(xa) == segment_length:
                segments.append([xa, ya, za
#                                 xg, yg, zg
                                ])
                labels.append(row['activity ID'])

    reshaped_segments = np.asarray(segments, dtype= np.float32).reshape(-1, segment_length, NUM_FEATURES)
    labels = np.asarray(labels)
    
    return reshaped_segments, labels

### Spliting the data
The researchers opted for a window size of 100 without any overlaps. Furthermore, the data was split into 70% training and 30% testing samples 

In [ ]:
labels_path = 'HAPT Data Set/RawData/labels.txt'
exp_data = read_experiments(labels_path)

SEGMENT_LENGTH = 100
STEP = 100
activity_labels_path = 'HAPT Data Set/activity_labels.txt' 
LABELS = pd.read_csv(activity_labels_path, delimiter = ' ', header = None)[1].tolist()

segments,labels = segment_experiments(exp_data,SEGMENT_LENGTH,STEP)

divider = int(len(segments) * .7)

x_train = segments[:divider]
y_train = labels[:divider]

x_test = segments[divider:]
y_test = labels[divider:]

num_time_periods, dims = x_train.shape[1], x_train.shape[2]
num_classes = len(LABELS)

input_shape = (num_time_periods * dims)
x_train = x_train.reshape(x_train.shape[0],input_shape)

y_train = y_train - 1

x_train = x_train.astype('float32')
y_train = y_train.astype('float32')


y_train = utils.to_categorical(y_train, num_classes)

# test data

x_test = x_test.reshape(x_test.shape[0],input_shape)

y_test = y_test - 1

x_test = x_test.astype('float32')
y_test = y_test.astype('float32')

y_test = utils.to_categorical(y_test, num_classes)


### Training
All models where trained with the same parameters - A batch size of 400 samples, on 50 Epochs, using the Adam optimizer and categorical cross entropy for the loss function.

In [28]:
def train(model):
    # The EarlyStopping callback monitors training accuracy:
    # if it fails to improve for two consecutive epochs,
    # training stops early
    callbacks_list = [
        tf.keras.callbacks.ModelCheckpoint(
            filepath='cnn_best_model.{epoch:02d}-{val_loss:.2f}.h5',
            monitor='val_loss', save_best_only=True),
        tf.keras.callbacks.EarlyStopping(monitor='acc', patience=50),
        tf.keras.callbacks.TensorBoard(log_dir='tfb_logs', histogram_freq=0,
              write_graph=True, write_images=True),
    ]

    model.compile(loss='categorical_crossentropy',
                    optimizer='adam', metrics=['accuracy'])

    BATCH_SIZE = 400
    EPOCHS = 50

    # Enable validation to use ModelCheckpoint and EarlyStopping callbacks.
    history = model.fit(x_train,
                          y_train,
                          batch_size=BATCH_SIZE,
                          epochs=EPOCHS,
                          callbacks=callbacks_list,
                          validation_split=0.2,
                          verbose=1)


### Using TFLite for RNNs
As of the moment, TFLite(which is used for run Tensorflow models on mobile) does not support LSTMs. To address this the researchers use the experimental build of the Tensorflow. The following function creates LSTM layers :

In [ ]:
def buildLstmLayer(inputs, num_layers, num_units):
    """Build the lstm layer.

    Args:
    inputs: The input data.
    num_layers: How many LSTM layers do we want.t
    num_units: The unmber of hidden units in the LSTM cell.
    """
    lstm_cells = []
    for i in range(num_layers):
        lstm_cells.append(
            tf.lite.experimental.nn.TFLiteLSTMCell(
                num_units, forget_bias=0, name='rnn{}'.format(i)))
    lstm_layers = tf.keras.layers.StackedRNNCells(lstm_cells)
    # Assume the input is sized as [batch, time, input_size], then we're going
    # to transpose to be time-majored.
    transposed_inputs = tf.transpose(inputs, perm=[1, 0, 2])
    outputs, _ = tf.lite.experimental.nn.dynamic_rnn(
        lstm_layers,
        transposed_inputs,
        dtype='float32',
        time_major = True)
    unstacked_outputs = tf.unstack(outputs, axis=0)
    return unstacked_outputs[-1]

## CNN MODEL
The first model that was unsed in this research was a CNN model with the following architecture:
![CNN Model](../images/cnn_model.png)

In [ ]:
cnn_model = tf.keras.Sequential()
cnn_model.add( Reshape( (SEGMENT_LENGTH, dims), input_shape=(input_shape,), name='cnn_input' ) )
cnn_model.add( Conv1D(100, 10, activation='relu', input_shape = (SEGMENT_LENGTH,dims)) )
cnn_model.add( Conv1D(100, 10, activation='relu') )
cnn_model.add( Conv1D(100, 10, activation='relu') )
cnn_model.add( MaxPooling1D(3) )
cnn_model.add( Dropout(0.5) )
cnn_model.add( Conv1D(160, 10, activation='relu') )
cnn_model.add( Conv1D(160, 10, activation='relu') )
cnn_model.add( GlobalAveragePooling1D() )
cnn_model.add( Dense(num_classes, activation = tf.nn.softmax ,name='cnn_output') )
print(cnn_model.summary())

## RNN Model
The RNN model uses LSTM cells and follows the following high level architectural design:
![RNN Model](../images/rnn_model.png)

In [ ]:
tf.reset_default_graph()
rnn_model = tf.keras.Sequential()
rnn_model.add( Reshape((SEGMENT_LENGTH, dims), input_shape = (input_shape,)) )
rnn_model.add( Input(shape=(SEGMENT_LENGTH, dims) ) )
rnn_model.add( Lambda(buildLstmLayer, arguments={'num_layers' : 2, 'num_units' : 64}))
rnn_model.add( Dropout(0.5) )
rnn_model.add( Dense(100, activation = 'relu'))
rnn_model.add( Dense(num_classes, activation = 'softmax') )
print(rnn_model.summary())

## CNN-RNN Hybrid
The hybrid model is a combination of multiple time distributed CNN layers and RNN layers using LSTMs. The model follows the following high level architectural design:
![RNN Model](../images/cnn_rnn_model.png)

In [ ]:
SUB_STEPS = 5
SUB_LENGTH = 20

tf.reset_default_graph()

cnn_rnn_model = tf.keras.Sequential()
# cnn_rnn_model.add( Reshape((SEGMENT_LENGTH, dims), input_shape = (input_shape,)) )
cnn_rnn_model.add( Reshape((SUB_STEPS,SUB_LENGTH, dims), input_shape = (input_shape,)) )
cnn_rnn_model.add( TimeDistributed(Conv1D(filters = 100, kernel_size = 3,  activation='relu'), input_shape = (None, SUB_LENGTH, dims)))
cnn_rnn_model.add( TimeDistributed(Conv1D(filters = 100, kernel_size = 3,  activation='relu')) )
cnn_rnn_model.add( TimeDistributed(Dropout(0.5)) )
cnn_rnn_model.add( TimeDistributed(MaxPooling1D(pool_size=2)) )
cnn_rnn_model.add( (TimeDistributed(Flatten())) )
cnn_rnn_model.add( Lambda(buildLstmLayer, arguments={'num_layers' : 2, 'num_units' : 100}) )
cnn_rnn_model.add( Dropout(0.5) )
cnn_rnn_model.add( Dense(100, activation = 'relu'))
cnn_rnn_model.add( Dense(num_classes, activation = 'softmax') )
print(cnn_rnn_model.summary())

## Running Experiments
The researchers ran experiments wherein each model was retrained 10 times for each test scenario (accelerometer only vs accelerometer + gyroscope)

In [ ]:
train(cnn_model)

### test model

In [ ]:
score = cnn_model.evaluate(x_test, y_test, verbose=1)

print("\nAccuracy on test data: %0.2f" % score[1])
print("\nLoss on test data: %0.2f" % score[0])

### test tflite

In [ ]:
path = 'CNN_2.tflite'
interpreter = tf.lite.Interpreter(model_path=path)
interpreter.get_input_details()

x_test = np.array([np.genfromtxt('test.txt',delimiter = ',')]).astype('float32')

try:
    interpreter.allocate_tensors()
except ValueError:
    assert False

MINI_BATCH_SIZE = 1
correct_case = 0
for i in range(len(x_test)):
    input_index = (interpreter.get_input_details()[0]['index'])
    interpreter.set_tensor(input_index, x_test[i * MINI_BATCH_SIZE: (i + 1) * MINI_BATCH_SIZE])
    interpreter.invoke()
    output_index = (interpreter.get_output_details()[0]['index'])
    result = interpreter.get_tensor(output_index)
    # Reset all variables so it will not pollute other inferences.
    interpreter.reset_all_variables()
    # Evaluate.
    print(result)
    prediction = np.argmax(result)
    if prediction == np.argmax(y_test[i]):
        correct_case += 1

print('TensorFlow Lite Evaluation result is {}'.format(correct_case * 1.0 / len(x_test)))

## References
[1] Jorge-L. Reyes-Ortiz, Luca Oneto, Albert Samà, Xavier Parra, Davide Anguita. Transition-Aware Human Activity Recognition Using Smartphones. Neurocomputing. Springer 2015.